In [35]:
import numba as nb
from numba import prange

In [36]:
# считываение из файла
def read_input(file_str: str):
    with open(file_str) as f:

        lines = f.readlines()
        lines.reverse()

        n = int(lines.pop().split()[0])
        parents = [int(s) for s in lines.pop().split(' ')]
        values = [int(s) for s in lines.pop().split(' ')]

        diseases = []
        num_of_diseases = int(lines.pop().split()[0])
        for i in range(num_of_diseases):
            dis = lines.pop()
            diseases.append([int(s) for s in dis.split(' ')])

        patients = []
        num_of_patients = int(lines.pop().split()[0])
        for i in range(num_of_patients):
            dis = lines.pop()
            patients.append([int(s) for s in dis.split(' ')])

        # return [n, relationships, values, diseases, patients]
        return {
            "n": n,
            "parents": parents,
            "values": values,
            "diseases": diseases,
            "patients": patients,
        }


data = read_input('./tests/test1')
n = data["n"]
parents = data["parents"]
values = data["values"]
diseases = data["diseases"]
patients = data["patients"]
print(n)

19830


In [37]:
from collections import defaultdict

parents_hmap = defaultdict(int)  # child: parent
children_hmap = defaultdict(list)  # parent: [children]
values_hmap = defaultdict(int)  # node: value
is_checked_hmap = defaultdict(bool)  # node: value

values_hmap[1] = values[0]
is_checked_hmap[1] = False
for i in range(len(parents)):
    child_index = i + 2
    parent_index = parents[i]

    parents_hmap[child_index] = parent_index
    children_hmap[parent_index].append(child_index)
    values_hmap[child_index] = values[i + 1]
    is_checked_hmap[child_index] = False

# print(parents_hmap)
# print(children_hmap)
# print(values_hmap)
# print(is_checked_hmap)

In [38]:
depth_hmap = defaultdict(list)  # parent: [all children in subtree]


def fill_depth_hmap(cur, index):
    depth_hmap[cur] = index
    for child in children_hmap[cur]:
        fill_depth_hmap(child, index + 1)


fill_depth_hmap(1, 0)
print(depth_hmap)

defaultdict(<class 'list'>, {1: 0, 2: 1, 3: 1, 10087: 2, 12930: 3, 14784: 2, 4: 1, 7970: 2, 8600: 2, 5: 1, 4200: 2, 5434: 2, 6: 1, 2931: 2, 8471: 2, 7: 1, 3843: 2, 5274: 2, 8: 1, 3923: 2, 5888: 2, 9: 1, 615: 2, 10: 1, 9643: 2, 11: 1, 609: 2, 16540: 3, 7752: 2, 10636: 3, 17164: 4, 19410: 3, 19699: 3, 15435: 2, 12: 1, 1181: 2, 5300: 2, 13: 1, 1911: 2, 5306: 2, 14: 1, 4198: 2, 17939: 2, 15: 1, 16: 1, 3709: 2, 12307: 3, 15009: 3, 19033: 3, 4057: 2, 17123: 3, 17: 1, 4588: 2, 5774: 2, 18: 1, 4803: 2, 4804: 2, 10169: 3, 16801: 3, 19300: 3, 19: 1, 1172: 2, 10614: 3, 13721: 2, 17337: 3, 20: 1, 6862: 2, 7432: 2, 10341: 3, 13008: 4, 21: 1, 22: 1, 4345: 2, 14329: 3, 15130: 3, 16264: 3, 19044: 4, 4627: 2, 10022: 2, 12571: 2, 16838: 2, 23: 1, 1913: 2, 6226: 2, 12766: 2, 17045: 3, 24: 1, 484: 2, 5135: 2, 25: 1, 85: 2, 26: 1, 7564: 2, 27: 1, 28: 1, 2157: 2, 2837: 2, 29: 1, 5667: 2, 30: 1, 31: 1, 4620: 2, 9133: 2, 32: 1, 770: 2, 6867: 2, 33: 1, 1617: 2, 17841: 3, 34: 1, 1450: 2, 35: 1, 36: 1, 37: 1, 59

In [39]:
@nb.njit()
def find_common_parent(i1, i2):
    depth1 = depth_hmap[i1]
    depth2 = depth_hmap[i2]
    if depth1 < depth2:
        depth1, depth2 = depth2, depth1
        i1, i2 = i2, i1

    while depth1 > depth2:
        depth1 -= 1
        i1 = parents_hmap[i1]

    while i1 != i2:
        depth1 -= 1
        i1 = parents_hmap[i1]
        depth2 -= 1
        i2 = parents_hmap[i2]
    return values_hmap[i1]

In [40]:
@nb.njit(parallel=True)
def foo():
    for i in range(len(patients)):
        patient = patients[i]
        IC_LCA_sum_max = 0
        best_k = -1
        for k in prange(len(diseases)):
            disease = diseases[k]
            IC_LCA_sum = 0
            for p in patient:
                IC_LCA_max = 0
                for d in disease:
                    IC_LCA = find_common_parent(p, d)
                    IC_LCA_max = max(IC_LCA_max, IC_LCA)
                IC_LCA_sum += IC_LCA_max
            if IC_LCA_sum > IC_LCA_sum_max:
                IC_LCA_sum_max = IC_LCA_sum
                best_k = k
            # IC_LCA_sum_max = max(IC_LCA_sum, IC_LCA_sum_max)
        print(i, best_k + 1)


foo()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'depth_hmap':[0m [1m[1mCannot determine Numba type of <class 'collections.defaultdict'>[0m
[1m
File "../../../../../var/folders/hh/8pgkyy797f7941n9r5qyp10w0000gn/T/ipykernel_34788/840456041.py", line 3:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m
[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function find_common_parent at 0x109a39d00>))[0m
[0m[1mDuring: typing of call at /var/folders/hh/8pgkyy797f7941n9r5qyp10w0000gn/T/ipykernel_34788/3470530339.py (13)
[0m
[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function find_common_parent at 0x109a39d00>))[0m
[0m[1mDuring: typing of call at /var/folders/hh/8pgkyy797f7941n9r5qyp10w0000gn/T/ipykernel_34788/3470530339.py (13)
[0m
[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function find_common_parent a

In [227]:
subtree_hmap = defaultdict(list)  # parent: [all children in subtree]


def fill_subtrees(cur):
    children = children_hmap[cur]
    subtree_hmap[cur].append(cur)
    if len(children) == 0:
        return [cur]

    for child in children:
        for el in fill_subtrees(child):
            subtree_hmap[cur].append(el)
    return subtree_hmap[cur]


fill_subtrees(1)
# print(subtree_hmap)

[1,
 2,
 3,
 10087,
 12930,
 14784,
 4,
 7970,
 8600,
 5,
 4200,
 5434,
 6,
 2931,
 8471,
 7,
 3843,
 5274,
 8,
 3923,
 5888,
 9,
 615,
 10,
 9643,
 11,
 609,
 16540,
 7752,
 10636,
 17164,
 19410,
 19699,
 15435,
 12,
 1181,
 5300,
 13,
 1911,
 5306,
 14,
 4198,
 17939,
 15,
 16,
 3709,
 12307,
 15009,
 19033,
 4057,
 17123,
 17,
 4588,
 5774,
 18,
 4803,
 4804,
 10169,
 16801,
 19300,
 19,
 1172,
 10614,
 13721,
 17337,
 20,
 6862,
 7432,
 10341,
 13008,
 21,
 22,
 4345,
 14329,
 15130,
 16264,
 19044,
 4627,
 10022,
 12571,
 16838,
 23,
 1913,
 6226,
 12766,
 17045,
 24,
 484,
 5135,
 25,
 85,
 26,
 7564,
 27,
 28,
 2157,
 2837,
 29,
 5667,
 30,
 31,
 4620,
 9133,
 32,
 770,
 6867,
 33,
 1617,
 17841,
 34,
 1450,
 35,
 36,
 37,
 5965,
 9612,
 15411,
 38,
 7860,
 11592,
 17022,
 13093,
 14927,
 17218,
 14373,
 39,
 6497,
 9489,
 40,
 41,
 4185,
 17907,
 42,
 8573,
 43,
 856,
 44,
 45,
 5474,
 6699,
 46,
 3518,
 13153,
 15696,
 15889,
 47,
 1262,
 2442,
 14695,
 48,
 2731,
 10697,
 49

In [ ]:
common_parent_hmap = defaultdict()


def fill_common_parent_hmap(cur):
    # subtree = subtree_hmap[cur]
    subtrees = []
    for child in children_hmap[cur]:
        subtrees.append(subtree_hmap[child])
    for subtree1 in subtrees:
        for subtree2 in subtrees:
            if subtree1 != subtree2:
                for el1 in subtree1:
                    for el2 in subtree2:
                        common_parent_hmap[el1, el2] = values_hmap[cur]
                        common_parent_hmap[el2, el1] = values_hmap[cur]
    for el in subtree_hmap[cur]:
        common_parent_hmap[cur, el] = values_hmap[cur]
        common_parent_hmap[el, cur] = values_hmap[cur]
    for child in children_hmap[cur]:
        fill_common_parent_hmap(child)


fill_common_parent_hmap(1)
# print(common_parent_hmap)

In [220]:
import numpy as np

for patient in patients:
    IC_LCA_sum_max = 0
    best_k = -1
    for k in range(len(diseases)):
        disease = diseases[k]
        IC_LCA_sum = 0
        for p in patient:
            IC_LCA_max = 0
            for d in disease:
                IC_LCA = common_parent_hmap[p, d]
                IC_LCA_max = max(IC_LCA_max, IC_LCA)
            IC_LCA_sum += IC_LCA_max
        if IC_LCA_sum > IC_LCA_sum_max:
            IC_LCA_sum_max = IC_LCA_sum
            best_k = k
        # IC_LCA_sum_max = max(IC_LCA_sum, IC_LCA_sum_max)
    print(best_k + 1)

2
1
2
2
